In [1]:
with open("verwandlung.txt", "r", encoding="utf-8") as file:
    contents = file.read()
    
contents = "\n".join(contents.split("\n")[59:1952])

In [2]:
# http://www.nltk.org/

import nltk
from nltk import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer

nltk.download('punkt')

tokens = word_tokenize(contents)

cv = CountVectorizer(max_features=1000, lowercase=False, token_pattern="(.*)")
cv.fit(tokens)

features = cv.get_feature_names()

word_to_int = {}
int_to_word = {}

for i in range(0, len(features)):
    word = features[i]
    
    word_to_int[word] = i
    int_to_word[i] = word

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
tokens_transformed = [word_to_int[word] for word in tokens if word in word_to_int]

In [7]:
import numpy as np

X = []
y = []

seq_length = 40

for i in range(0, len(tokens_transformed) - seq_length):
    X.append(tokens_transformed[i:i+seq_length])
    y.append(tokens_transformed[i+seq_length])
    
X = np.array(X)
y = np.array(y)

In [14]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Embedding, Flatten

model = Sequential()
model.add(Embedding(cv.max_features, 100, input_shape=(seq_length,)))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(128, return_sequences=False))

model.add(Dense(cv.max_features, activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [21]:
from keras.utils import to_categorical

model.fit(X, to_categorical(y, num_classes=cv.max_features), epochs=10, batch_size=32)

Epoch 1/10
18748/18748 [==============================] - 81s 4ms/step - loss: 5.5045 - acc: 0.0993
Epoch 2/10
18748/18748 [==============================] - 82s 4ms/step - loss: 5.3573 - acc: 0.0994
Epoch 3/10
18748/18748 [==============================] - 80s 4ms/step - loss: 5.2184 - acc: 0.1001
Epoch 4/10
18748/18748 [==============================] - 81s 4ms/step - loss: 5.0821 - acc: 0.1131
Epoch 5/10
18748/18748 [==============================] - 80s 4ms/step - loss: 4.9574 - acc: 0.1243
Epoch 6/10
18748/18748 [==============================] - 80s 4ms/step - loss: 4.8484 - acc: 0.1297
Epoch 7/10
18748/18748 [==============================] - 80s 4ms/step - loss: 4.7785 - acc: 0.1336
Epoch 8/10
18748/18748 [==============================] - 80s 4ms/step - loss: 4.7136 - acc: 0.1382
Epoch 9/10
18748/18748 [==============================] - 80s 4ms/step - loss: 4.5984 - acc: 0.1436
Epoch 10/10
18748/18748 [==============================] - 80s 4ms/step - loss: 4.5066 - acc: 0.1469

In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 40, 100)           100000    
_________________________________________________________________
lstm_7 (LSTM)                (None, 40, 128)           117248    
_________________________________________________________________
lstm_8 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_3 (Dense)              (None, 1000)              129000    
Total params: 477,832
Trainable params: 477,832
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.save("verwandlung.model")

In [24]:
import pickle

with open("word_to_int.pickle", "wb") as file:
    pickle.dump(word_to_int, file)
    
    
with open("int_to_word.pickle", "wb") as file:
    pickle.dump(int_to_word, file)